# Shift Convolution (Shift-Conv)

Bu notebook, **Shift Convolution (Shift-Conv)** yöntemini en temelden başlayarak; **neden ortaya çıktığını**, **tam olarak ne yaptığını**, **hangi bloklarla birlikte kullanıldığını** ve **PyTorch ile nasıl uygulanacağını** detaylı şekilde anlatır.

## Hedef
- Shift işleminin (kanal bazlı kaydırma) tanımını görmek
- Konvolüsyonun uzamsal karıştırmasını nasıl “ucuz” şekilde sağladığını anlamak
- **Shift + 1×1 Conv** kombinasyonunun neden kritik olduğunu kavramak
- Parametre/FLOP karşılaştırması yapmak
- PyTorch ile **tam çalışan** örnek kodlar görmek


## 1) Motivasyon: Neden Shift-Conv?

Mobil/edge (telefon, Jetson, gömülü) ağlarda hedef şudur:

- **FLOP** (hesap yükü) azalt
- **Parametre** azalt
- Yine de uzamsal (spatial) özellikleri iyi yakala

Klasik **K×K** konvolüsyonun maliyeti yüksektir.

### Klasik Conv2d maliyeti (yaklaşık)
- Parametre: `C_in * C_out * K * K`
- FLOP (yaklaşık): `H * W * C_in * C_out * K * K`

### Shift fikri
**K×K konvolüsyonun yaptığı uzamsal karıştırmayı** (komşu piksellere bakma) parametresiz bir operasyonla yap:

- Kanalların bir kısmını **sağa/sola/yukarı/aşağı** kaydır
- Sonra **1×1 Conv** ile kanalları karıştır (learnable mixing)

Bu yaklaşım, K×K conv yerine:

1) `shift` (parametresiz, FLOP çok düşük)
2) `1×1 conv` (öğrenilebilir ama K×K değil, daha ucuz)

kombinasyonunu kullanır.


## 2) Shift İşlemi Nedir?

Shift işlemi, her kanalı (veya kanal gruplarını) belirli bir yönde kaydırır.

Örnek: 3×3 komşuluk yönleri

- yukarı (0, -1)
- aşağı (0, +1)
- sol (-1, 0)
- sağ (+1, 0)
- (opsiyonel) çaprazlar ve merkez (0,0)

### En kritik nokta
Shift operasyonu **öğrenilebilir ağırlık içermez**.

- Parametre sayısı: **0**
- Ama uzamsal bilgi taşır: komşu pikselleri kanallara dağıtır

Bu yüzden Shift, tek başına konvolüsyonun yerini tam alamaz.
Çünkü konvolüsyonun “öğrenme” kısmı ağırlıklardadır.

Shift-Conv yaklaşımında öğrenme çoğunlukla **1×1 conv** ile yapılır:

> `Shift (spatial mixing) + 1×1 Conv (channel mixing)`


## 3) Shift-Conv Bloğu: Neden 1×1 Conv ile birlikte?

Shift şunu yapar:
- Kanalların bazılarını sağa/sola/yukarı/aşağı kaydırır
- Böylece farklı uzamsal konumlardan bilgi gelir

Ama shift:
- Kanallar arasında **öğrenilebilir bir karışım** yapmaz
- Sadece yeniden düzenler

**1×1 Conv** şunu yapar:
- Kanallar arasında öğrenilebilir mixing sağlar
- Parametre: `C_in * C_out`
- FLOP: `H * W * C_in * C_out`

Bu yüzden Shift-Conv genelde şu bloktur:

```
x → Shift → 1×1 Conv → (BN) → (Activation)
```

Bu, **Depthwise Separable Conv** ile zihinsel olarak benzer bir ayrıştırmadır:
- Depthwise: spatial mixing (kanal başına)
- Pointwise: channel mixing

Shift: spatial mixing’i daha da ucuz (parametresiz) hale getirir.


## 4) Shift Nasıl Tasarlanır? (Kanal Dağıtımı)

Pratikte kanalları yönlere bölüştürürsün.

Örneğin `C` kanalın olsun ve 5 yön kullan:
- (0,0) merkez (shift yok)
- (1,0) sağ
- (-1,0) sol
- (0,1) aşağı
- (0,-1) yukarı

Kanalların yaklaşık `C/5`’i her yöne atanır.

Not:
- Sınırda taşan değerler genelde **zero padding** ile doldurulur.
- Alternatif: `padding_mode='replicate'` gibi seçenekler (ama shift basit olsun diye çoğu implementasyon zero kullanır).


## 5) PyTorch ile Shift Implementasyonu (Tam Çalışan)

Aşağıdaki implementasyon:
- `x` tensorunu (B, C, H, W) alır
- Kanalları gruplara böler
- Her grubu farklı yöne **torch.roll** ile kaydırır
- Sınırdaki wrap-around etkisini engellemek için sıfırlama uygular (zero padding benzeri)

Bu sayede shift işlemi **deterministik ve parametresiz** olur.


In [1]:
import torch
import torch.nn as nn


def shift2d(x: torch.Tensor, directions=None) -> torch.Tensor:
    """Parametresiz shift işlemi.

    Args:
        x: (B, C, H, W)
        directions: kanal gruplarına atanacak (dx, dy) listesi.
            Varsayılan: merkez + 4 yön (5 grup)

    Not:
        torch.roll wrap-around yapar. Biz wrap-around bölgelerini sıfırlayarak
        zero-padding etkisi oluşturuyoruz.
    """
    if directions is None:
        directions = [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]  # center, right, left, down, up

    B, C, H, W = x.shape
    G = len(directions)

    base = C // G
    sizes = [base] * (G - 1) + [C - base * (G - 1)]
    xs = torch.split(x, sizes, dim=1)

    out_chunks = []
    for chunk, (dx, dy) in zip(xs, directions):
        if dx == 0 and dy == 0:
            out_chunks.append(chunk)
            continue

        rolled = torch.roll(chunk, shifts=(dy, dx), dims=(-2, -1))

        # wrap-around sıfırlama
        if dy > 0:
            rolled[..., :dy, :] = 0
        elif dy < 0:
            rolled[..., dy:, :] = 0

        if dx > 0:
            rolled[..., :, :dx] = 0
        elif dx < 0:
            rolled[..., :, dx:] = 0

        out_chunks.append(rolled)

    return torch.cat(out_chunks, dim=1)


# Hızlı test
x = torch.randn(2, 10, 8, 8)
y = shift2d(x)
print('x:', x.shape, 'y:', y.shape)


x: torch.Size([2, 10, 8, 8]) y: torch.Size([2, 10, 8, 8])


## 6) Shift-Conv Blok: Shift + 1×1 Conv

Şimdi shift fonksiyonunu bir modüle sarıyoruz ve arkasına `1×1 Conv` ekliyoruz.

Bu blok, pratikte K×K conv yerine kullanılabilir.


In [2]:
class ShiftConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, directions=None, bias=False):
        super().__init__()
        self.directions = directions
        self.pw = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = shift2d(x, self.directions)
        x = self.pw(x)
        x = self.bn(x)
        x = self.act(x)
        return x

# Test
model = ShiftConvBlock(32, 64)
inp = torch.randn(4, 32, 64, 64)
out = model(inp)
print('inp:', inp.shape, 'out:', out.shape)


inp: torch.Size([4, 32, 64, 64]) out: torch.Size([4, 64, 64, 64])


## 7) Parametre ve FLOP Karşılaştırması (Hızlı Hesap)

Bir katman karşılaştırması yapalım:

- Klasik `3×3 Conv`: Param = `C_in*C_out*9`
- Shift-Conv: Param = `C_in*C_out` (sadece 1×1)

Bu, özellikle `K=3` için teorik olarak ~9× parametre farkı demek.

Şimdi sayısal görelim.


In [3]:
def params_conv2d(cin, cout, k):
    return cin * cout * k * k

def params_shift_conv(cin, cout):
    return cin * cout  # shift parametresiz

cin, cout, k = 64, 128, 3
p_conv = params_conv2d(cin, cout, k)
p_shift = params_shift_conv(cin, cout)
print('3x3 Conv params:', p_conv)
print('Shift+1x1 params:', p_shift)
print('Param ratio (Conv / Shift):', p_conv / p_shift)


3x3 Conv params: 73728
Shift+1x1 params: 8192
Param ratio (Conv / Shift): 9.0


## 8) Nerede Kullanılır? (Pratik Tasarım)

Shift-Conv özellikle:
- Mobil backbone bloklarında
- 3×3 conv’un çok maliyetli olduğu yerlerde

Şu şablonlarla kullanılır:

### Şablon A: Conv yerine Shift-Conv
```
3×3 Conv yerine:
Shift → 1×1 Conv
```

### Şablon B: Bottleneck benzeri
```
1×1 (expand) → Shift → 1×1 (project)
```

Bu yaklaşım, Inverted Bottleneck mantığına benzer şekilde kanal genişletme/sıkıştırma ile çalışır.


## 9) Mini Model Örneği: Shift-Conv Kullanan CNN

Aşağıdaki model:
- Basit bir stem conv
- Ardından 2 adet ShiftConvBlock
- Global average pooling + classification head

Bu, Shift-Conv’un gerçek kullanımını görmen için minimal bir örnek.


In [4]:
class SmallShiftCNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )

        self.block1 = ShiftConvBlock(32, 64)
        self.block2 = ShiftConvBlock(64, 128)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


# Test
m = SmallShiftCNN(in_channels=3, num_classes=10)
dummy = torch.randn(8, 3, 64, 64)
pred = m(dummy)
print('pred:', pred.shape)


pred: torch.Size([8, 10])


## 10) Sık Yapılan Hatalar ve İpuçları

1. **torch.roll wrap-around yapar**
   - Biz wrap-around bölgelerini sıfırladık.
   - Aksi halde sağa kaydırınca soldan değer taşar (sahte bilgi).

2. Shift tek başına yeterli değil
   - Öğrenme `1×1 conv` ile olur.

3. Kanal sayısı küçükse grup dağılımı dengesiz olabilir
   - Çok küçük C’de bazı yönlere çok az kanal düşer.
   - Bu durumda yön sayısını azaltabilir veya farklı dağıtım yapabilirsin.

4. Performans notu
   - Shift teorik olarak ucuzdur.
   - Ancak PyTorch’ta `roll + slice` bazı ortamlarda bellek hareketi yaratabilir.
   - Mobil inference için özel optimize edilmiş implementasyonlar daha hızlı olabilir.


## 11) Özet

- Shift-Conv, K×K konvolüsyonun uzamsal etkisini **parametresiz bir kaydırma** ile taklit eder.
- Öğrenilebilir kısım çoğunlukla **1×1 conv** ile sağlanır.
- Amaç: **parametre ve FLOP azaltmak** (özellikle mobil ağlar için).
- Tipik blok: `Shift → 1×1 Conv → BN → Activation`

Bu noktadan sonra istersen:
- Shift-Conv’u kendi CNN bloğuna (ör. Hyso) entegre edelim
- Aynı sahnede 3×3 conv vs shift+1×1 çıktıları görselleştirelim
- FLOP hesaplarını daha detaylı (gerçek H×W ile) çıkaralım
